In [1]:
import json
from retrieval.rank_dataset import retrieve_top_k_with_contriver
from asymmetric_baselines import sub_cats
from transformers import AutoModel, AutoTokenizer
import tqdm

/home/oyilmazel_umass_edu/.conda/envs/lamp/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
f_path = "data/processed/ae_processed_test.json"

with open(f_path, "r") as f:
    dataset = json.load(f)

In [3]:
len(dataset)

401

In [5]:
dataset = dataset
rank_dict = dict()

tokenizer = AutoTokenizer.from_pretrained("facebook/contriever-msmarco")
contriver = AutoModel.from_pretrained("facebook/contriever-msmarco").to("cuda:0")
contriver.eval()

dataset_new = []
for data in tqdm.tqdm(dataset):
    profile = data['profile']
    corpus = [x['text'] for x in profile]
    query = data['question']
    ids = [x['id'] for x in profile]
    
    ranked_profile = retrieve_top_k_with_contriver(contriver, tokenizer, corpus, profile, query, len(profile), 2)
    # ranked_profile = [{'id': ids[i], 'text': ranked_profile[i]['text']} for i in range(len(ranked_profile))]        
    ranked_profile = [ranked_profile[i] for i in range(len(ranked_profile))]        
    data['profile'] = ranked_profile
    dataset_new.append(data)


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [ ]:
def attach_domain(dataset, sub_cats):
    new_dataset = []
    for data in dataset:
        profile = data["profile"]
        categories = [post["category"] for post in profile]
        domains = []

        for category in categories:
            if category in sub_cats["ae"]:
                domains.append("ae")
            elif category in sub_cats["lp"]:
                domains.append("lp")
            elif category in sub_cats["sc"]:
                domains.append("sc")
        
        data["domains"] = domains
        new_dataset.append(data)
    
    return new_dataset

def precision_at_k(ranked_list, relevant_domain, k):
    count = 0
    for domain in ranked_list[:k]:
        if domain == relevant_domain:
            count += 1
    
    return count / k

In [ ]:
dataset_new = attach_domain(dataset_new, sub_cats)

In [ ]:
# Macro
target = "ae"
k = 5

running_precision = 0
for i in range(len(dataset_new)):
    running_precision += precision_at_k(dataset_new[i]["domains"], target, k)
macro_precision = running_precision / len(dataset_new)

print(f"Macro Precision: {macro_precision:.4f}")